# 🏭 OOP w ML: Fabryki i Narzędzia (Classmethod & Staticmethod)

Kiedy budujesz system ML, Twoje klasy (np. `ModelHandler` czy `DataLoader`) robią się skomplikowane.
Python daje nam dwa potężne dekoratory, żeby utrzymać czystość kodu:

1.  **`@staticmethod` (Narzędzie):**
    *   To funkcja, która siedzi wewnątrz klasy, ale **nie używa `self`**.
    *   Nie ma dostępu do danych obiektu.
    *   Służy jako funkcja pomocnicza (np. `clean_text`, `normalize_data`), którą trzymamy w klasie dla porządku.

2.  **`@classmethod` (Fabryka / Alternatywny Konstruktor):**
    *   Zamiast `self`, przyjmuje `cls` (klasę).
    *   Służy do tworzenia obiektów w nietypowy sposób (np. wczytanie z pliku, z bazy danych).
    *   To standardowy sposób na robienie `from_json`, `from_csv` itp.

W tym notatniku zbudujemy klasę `TextClassifier`, która używa obu tych mechanizmów.

In [1]:
import json

# Wyobraźmy sobie prostą klasę do obsługi modelu
class TextClassifier:
    
    # 1. Standardowy Konstruktor (__init__)
    def __init__(self, model_name, vocabulary, version=1):
        self.model_name = model_name
        self.vocabulary = vocabulary
        self.version = version
        print(f"🔧 Zainicjowano {self.model_name} (v{self.version})")

    # 2. ZWYKŁA METODA (Używa 'self')
    # Musi mieć dostęp do 'self.vocabulary', żeby działać
    def predict(self, text):
        # Najpierw używamy metody statycznej do czyszczenia
        clean_txt = TextClassifier.clean_input(text)
        
        # Symulacja predykcji
        score = len(clean_txt) / 100
        return f"Klasa: Pozytywna (Score: {score:.2f})"

    # 3. METODA STATYCZNA (@staticmethod)
    # Nie potrzebuje 'self'. Nie obchodzi jej, jaki to model.
    # Robi prostą robotę: string -> string.
    @staticmethod
    def clean_input(text):
        return text.lower().strip().replace("!", "")

    # 4. METODA KLASOWA (@classmethod) - FABRYKA
    # Służy do stworzenia obiektu na podstawie pliku JSON (słownika)
    # Zamiast 'self', dostaje 'cls' (czyli klasę TextClassifier)
    @classmethod
    def from_config(cls, config_dict):
        # Wyciągamy dane ze słownika
        name = config_dict.get('name', 'Unknown')
        vocab = config_dict.get('vocab', [])
        
        # Tworzymy i zwracamy obiekt (to samo co: return TextClassifier(...))
        return cls(model_name=name, vocabulary=vocab)
    
    # Druga fabryka - Wersja Produkcyjna (Domyślna)
    @classmethod
    def create_production(cls):
        return cls(model_name="Prod_BERT_v2", vocabulary=["tak", "nie"], version=99)

print("Klasa zdefiniowana.")

Klasa zdefiniowana.


## Test 1: Static Method (Narzędzie)

Zauważ, że możemy użyć `clean_input` na dwa sposoby:
1.  Z poziomu obiektu.
2.  Z poziomu samej klasy (nawet nie tworząc obiektu!). To bardzo wygodne.

In [2]:
raw_text = "  Hura! To działa!  "

# Użycie bez tworzenia obiektu (Jak zwykła funkcja z biblioteki)
cleaned = TextClassifier.clean_input(raw_text)

print(f"Oryginał: '{raw_text}'")
print(f"Po czyszczeniu: '{cleaned}'")

Oryginał: '  Hura! To działa!  '
Po czyszczeniu: 'hura to działa'


## Test 2: Class Method (Alternatywne Konstruktory)

To jest ten moment, gdzie Twój kod staje się "Pro".
Zamiast kazać użytkownikowi ręcznie wpisywać parametry do `__init__`, dajesz mu wygodne metody `from_...`.

In [3]:
# SCENARIUSZ A: Ręczne tworzenie (Standard)
model_1 = TextClassifier("Mój_Model", ["a", "b"])

# SCENARIUSZ B: Tworzenie z konfiguracji (np. pobranej z API)
config_from_api = {
    "name": "GPT-Mini",
    "vocab": ["hello", "world"],
    "ignore_this": 123
}

# Używamy naszej fabryki
model_2 = TextClassifier.from_config(config_from_api)

# SCENARIUSZ C: Szybki model produkcyjny (Defaults)
model_3 = TextClassifier.create_production()

print("-" * 30)
print(f"Model 1: {model_1.model_name}")
print(f"Model 2: {model_2.model_name}")
print(f"Model 3: {model_3.model_name} (Wersja {model_3.version})")

🔧 Zainicjowano Mój_Model (v1)
🔧 Zainicjowano GPT-Mini (v1)
🔧 Zainicjowano Prod_BERT_v2 (v99)
------------------------------
Model 1: Mój_Model
Model 2: GPT-Mini
Model 3: Prod_BERT_v2 (Wersja 99)


## 🧠 Podsumowanie: Kiedy czego używać?

Jako AI Engineer będziesz tego używać non-stop:

1.  **Zwykła metoda (`def predict(self)`):**
    *   Gdy funkcja musi znać stan obiektu (np. użyć wag modelu `self.weights`).

2.  **`@staticmethod`:**
    *   Gdy funkcja jest logicznie związana z klasą, ale jest niezależna od danych (np. `preprocess_image`, `remove_stopwords`).
    *   *Zasada:* Jeśli w metodzie ani razu nie używasz słowa `self`, to zrób ją statyczną. Będzie szybsza.

3.  **`@classmethod`:**
    *   Gdy chcesz stworzyć obiekt w nowy sposób (Konstruktor).
    *   Najczęstsze nazwy: `from_json`, `from_checkpoint`, `from_pretrained` (znasz to z HuggingFace? `BertModel.from_pretrained('bert-base')` to właśnie `@classmethod`!).